In [3]:
import numpy as np
from pygmo import fast_non_dominated_sorting as nds
from timeit import timeit
import pandas as pd
import numba

In [16]:
num_var = 10
num_samples=2000
mean = 0.5
std_dev = 0.1
means = [mean] * num_var
cov = np.eye(num_var) * np.square(std_dev)
data = np.random.multivariate_normal(means, cov, num_samples)

def setup_func(num_var, num_samples):
    mean = 0.5
    std_dev = 0.1
    means = [mean] * num_var
    cov = np.eye(num_var) * np.square(std_dev)
    data = np.random.multivariate_normal(means, cov, num_samples)
    return data

In [18]:
setup_func(10, 2000)

array([[0.46123709, 0.36933099, 0.59278853, ..., 0.54850498, 0.47636666,
        0.46197761],
       [0.31846639, 0.39835602, 0.57160771, ..., 0.58013561, 0.63216825,
        0.44818422],
       [0.4371848 , 0.5081325 , 0.35569554, ..., 0.45796313, 0.59656209,
        0.53850223],
       ...,
       [0.46274389, 0.41602239, 0.39930939, ..., 0.59483586, 0.46289174,
        0.60211574],
       [0.48400559, 0.52821663, 0.50622876, ..., 0.59286182, 0.75308299,
        0.60417046],
       [0.61557064, 0.33712149, 0.69187181, ..., 0.3118346 , 0.49577767,
        0.59196624]])

In [21]:
def setup_str(num_var, num_samples):
    return f"""
import numpy as np
from pygmo import fast_non_dominated_sorting as nds
import numba

num_var = {num_var}
num_samples= {num_samples}
mean = 0.5
std_dev = 0.1
means = [mean] * num_var
cov = np.eye(num_var) * np.square(std_dev)
data = np.random.multivariate_normal(means, cov, num_samples)

@numba.njit()
def dominates(x:np.ndarray, y:np.ndarray) -> bool:
    "Returns true if x dominates y"
    dom = False
    for i in range(len(x)):
        if x[i] > y[i]:
            return False
        elif x[i] < y[i]:
            dom = True
    return dom

@numba.njit()
def non_dominated(data:np.ndarray) -> np.ndarray:
    "Returns boolean array of same length as number of solutions. The value is true if corresponding solution is non-dominated. False otherwise"

    num_solutions = len(data)
    index = np.zeros(num_solutions, dtype=np.bool_)
    index[0] = True
    for i in range(1, num_solutions):
        index[i] = True
        for j in range(i):
            if not index[j]:
                continue
            if dominates(data[i], data[j]):
                index[j]=False
            elif dominates(data[j], data[i]):
                index[i]=False
                break
    return index

@numba.njit()
def fast_non_dominated_sort(data:np.ndarray) -> np.ndarray:
    "Returns n*f boolean array. n is the number of solutions, f is the number of fronts"

    num_solutions = len(data)
    indices = np.arange(num_solutions)
    taken = np.zeros(num_solutions, dtype=np.bool_)
    fronts = np.zeros((num_solutions, num_solutions), dtype=np.bool_)

    for i in indices:
        current_front = non_dominated(data[~taken])

        current_front_all = np.zeros(num_solutions, dtype=np.bool_)
        current_front_all[~taken] = current_front
        fronts[i] = current_front_all

        taken = taken + fronts[i]
        if not fronts[i].any():
            break
    return fronts[:i]

fast_non_dominated_sort(data)
    """

In [9]:
code1 = """nds(data)[0]"""

In [10]:
code2 = """fast_non_dominated_sort(data)"""

In [47]:
data = pd.DataFrame(columns=["num_samples", "num_obj", "pygmo", "numba"], dtype=float)

num_samples = [100, 200, 500, 1000, 2000, 3000, 5000, 10000]
num_objs = [3, 4, 5, 6, 7, 8, 9, 10, 20]

for num_sample in num_samples:
    print(num_sample)
    for num_obj in num_objs:
        setup = setup_str(num_obj, num_sample)
        pygmo_time = timeit(code1, setup, number=10)
        numba_time = timeit(code2, setup, number=10)
        data_row = pd.DataFrame(
            [[num_sample, num_obj, pygmo_time, numba_time]], 
            columns=["num_samples", "num_obj", "pygmo", "numba"]
        )
        data = data.append(data_row, ignore_index=True)

100
200
500
1000
2000
3000
5000
10000


In [48]:
data.to_csv("numba_tests.csv")

In [32]:
@numba.njit()
def dominates(x:np.ndarray, y:np.ndarray) -> bool:
    "Returns true if x dominates y"
    dom = False
    for i in range(len(x)):
        if x[i] > y[i]:
            return False
        elif x[i] < y[i]:
            dom = True
    return dom

In [33]:
@numba.njit()
def non_dominated(data:np.ndarray) -> np.ndarray:
    "Returns boolean array of same length as number of solutions. The value is true if corresponding solution is non-dominated. False otherwise"
    
    num_solutions = len(data)
    index = np.zeros(num_solutions, dtype=np.bool_)
    index[0] = True
    for i in range(1, num_solutions):
        index[i] = True
        for j in range(i):
            if not index[j]:
                continue
            if dominates(data[i], data[j]):
                index[j]=False
            elif dominates(data[j], data[i]):
                index[i]=False
                break
    return index
            

In [34]:
@numba.njit()
def fast_non_dominated_sort(data:np.ndarray) -> np.ndarray:
    "Returns n*f boolean array. n is the number of solutions, f is the number of fronts"
    
    num_solutions = len(data)
    indices = np.arange(num_solutions)
    taken = np.zeros(num_solutions, dtype=np.bool_)
    fronts = np.zeros((num_solutions, num_solutions), dtype=np.bool_)
    
    for i in indices:
        current_front = non_dominated(data[~taken])

        current_front_all = np.zeros(num_solutions, dtype=np.bool_)
        current_front_all[~taken] = current_front
        fronts[i] = current_front_all

        taken = taken + fronts[i]
        if not fronts[i].any():
            break
    return fronts[:i]
        

In [43]:
data.shape

(5000, 10)

In [72]:
%timeit nds(setup_func(100, 20000))

4.44 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%timeit indices_from_bool(fast_non_dominated_sort(setup_func(10, 20000)))

1.88 s ± 54.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
fronts = fast_non_dominated_sort(setup_func(10, 2000))

In [68]:
indices_from_bool = lambda data: [np.where(data[i]) for i in range(len(data))]

In [75]:
num_samples = [100, 200, 500, 1000, 2000,3000, 5000]
num_objs = [3, 4, 5, 6, 7, 8, 9, 10]

for num_sample in num_samples:
    for num_obj in num_objs:
        data = setup_func(num_obj, num_sample)
        pygmonds = nds(data)[0]
        numbands = indices_from_bool(fast_non_dominated_sort(data))
        print("Success!")
        for i in range(len(pygmonds)):
            if not (numbands[i] == np.sort(pygmonds[i])).all():
                print(f"oh no @ {num_sample}, {num_obj}")

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


In [37]:
(np.where(numbands[i])[0] == np.sort(pygmonds[i])).all()

True